In [0]:
%pip install kaleido --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org
%pip install tabulate --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org

In [0]:
# Importing libraries
import pyspark
import pyspark.sql.functions as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff

In [0]:
# Number formatting
pd.set_option('display.float_format', '{:.2f}'.format)

In [0]:
df = pd.read_parquet('/Workspace/Users/daguilarromero@scor.com/Data_science_project/01-data/df.parquet')

# Filtering data
#pr_df_raw = df.query('uw_year == 2023')
pr_df_raw = df


In [0]:
# 1.0 Summary of data structure

# 1.1 n rows
print('Number of rows :', pr_df_raw.shape[0])

# 1.2 n columns
print('Number of columns:', pr_df_raw.shape[1])

# 1.3 Column type frequency df
print(pr_df_raw.dtypes.value_counts())



##### 2.0 NA Values

In [0]:
na_summary = (
    pd.DataFrame({'column_name': pr_df_raw.columns, 'na_n': pr_df_raw.apply(lambda x: sum(x.isnull()))})
    .query('na_n > 0')
    .sort_values('na_n', ascending=False)
    .assign(na_pct = lambda x: round(x.na_n / pr_df_raw.shape[0], 4))
)

display(na_summary)


In [0]:
# Dropping columns with na_pct > threshold
threshold = 0.2
cols_to_drop_bc_na = na_summary.query(f'na_pct > {threshold}').column_name.to_list()
pr_df = pr_df_raw.drop(cols_to_drop_bc_na, axis=1)

print(f'Number of columns dropped bc of na_pct > {threshold} :', len(cols_to_drop_bc_na), 'columns')

In [0]:
print(pr_df.dtypes.value_counts())

In [0]:
categorical_columns = [
    'ri_region',
    'ri_area',
    'geo_level',
    'reporting_country', 
    'lobn2',
    'lobn1',
    'per_event', 
    'per_risk', 
  #  'earned_premium',
  #  'retention',
  #  'limit',
  #  'ultimate_amount',
  #  'log_ultimate_amount'
    ]

print(pr_df.get(categorical_columns).describe(percentiles = [0.10, 0.25, 0.5, 0.75, 0.99]).to_markdown(floatfmt='.2f'))

In [0]:
# Boxplots for Log Ultimate Amount by lobn1 and colored by Main Nature
(
    px.box(data_frame = pr_df, x = 'ri_region', y = 'ultimate_amount')
    .update_xaxes(matches = None, tickangle=270)
    .update_layout(
        title_text= f'Boxplots for Ultimate Amount by Reinsurance region', # title of plot
        xaxis_title_text='Reinsurance region', # xaxis label
        yaxis_title_text='Ultimate Amount')
    )

In [0]:
# Boxplots for Log Ultimate Amount by lobn1 and colored by Main Nature
(
    px.box(data_frame = pr_df, x = 'ri_area', y = 'ultimate_amount')
    .update_xaxes(matches = None, tickangle=270)
    .update_layout(
        title_text= f'Boxplots for Ultimate Amount by Reinsurance area', # title of plot
        xaxis_title_text='Reinsurance area', # xaxis label
        yaxis_title_text='Ultimate Amount')
    )

In [0]:
# Boxplots for Log Ultimate Amount by lobn1 and colored by Main Nature
(
    px.box(data_frame = pr_df, x = 'geo_level', y = 'ultimate_amount')
    .update_xaxes(matches = None, tickangle=270)
    .update_layout(
        title_text= f'Boxplots for Ultimate Amount by Geographic level', # title of plot
        xaxis_title_text='Geographic level', # xaxis label
        yaxis_title_text='Ultimate Amount')
    )

In [0]:
# Boxplots for Log Ultimate Amount by lobn1 and colored by Main Nature
(
    px.box(data_frame = pr_df, x = 'lobn2', y = 'ultimate_amount')
    .update_xaxes(matches = None, tickangle=270)
    .update_layout(
        title_text= f'Boxplots for Ultimate Amount by Line of business level 2', # title of plot
        xaxis_title_text='Line of business level 2', # xaxis label
        yaxis_title_text='Ultimate Amount')
    )

In [0]:
# Boxplots for Log Ultimate Amount by lobn1 and colored by Main Nature
(
    px.box(data_frame = pr_df, x = 'lobn1', y = 'ultimate_amount')
    .update_xaxes(matches = None, tickangle=270)
    .update_layout(
        title_text= f'Boxplots for Ultimate Amount by Line of business level 1', # title of plot
        xaxis_title_text='Line of business level 1', # xaxis label
        yaxis_title_text='Ultimate Amount')
    )

In [0]:
# Boxplots for Log Ultimate Amount by lobn1 and colored by Main Nature
(
    px.box(data_frame = pr_df, x = 'per_event', y = 'ultimate_amount')
    .update_xaxes(matches = None, tickangle=270)
    .update_layout(
        title_text= f'Boxplots for Ultimate Amount by Per event category', # title of plot
        xaxis_title_text='Per event category', # xaxis label
        yaxis_title_text='Ultimate Amount')
    )

In [0]:
# Boxplots for Log Ultimate Amount by lobn1 and colored by Main Nature
(
    px.box(data_frame = pr_df, x = 'per_risk', y = 'ultimate_amount')
    .update_xaxes(matches = None, tickangle=270)
    .update_layout(
        title_text= f'Boxplots for Ultimate Amount by Per Risk', # title of plot
        xaxis_title_text='Per risk category', # xaxis label
        yaxis_title_text='Ultimate Amount')
    )

In [0]:
numerical_columns = [
    'earned_premium',
    'retention',
    'limit',
    'ultimate_amount'
    ]

print(pr_df.get(numerical_columns).describe(percentiles = [0.10, 0.25, 0.5, 0.75, 0.99]).to_markdown(floatfmt='.2f'))

In [0]:
# Density distributions all years
x_groups = ['earned_premium', 'retention' , 'limit' ,'ultimate_amount']
x = [pr_df.get(i) for i in x_groups]

(
    ff.create_distplot(x, x_groups, bin_size=0.1)
    .update_layout(
        title='Density distributions for numerical columns',
        template = 'none')
)


In [0]:
x_groups = ['earned_premium','ultimate_amount']
numeric_dfs = [pr_df.get([f'{i}']).assign(type = f'{i}').rename(columns={f'{i}':'value'}) for i in x_groups]

numeric_df = pd.concat(numeric_dfs, axis=0)


px.histogram(numeric_df, x='value', nbins=1000, color = 'type', marginal='box', histnorm='probability density', barmode='overlay')

In [0]:
x_groups = ['retention','limit']
numeric_dfs = [pr_df.get([f'{i}']).assign(type = f'{i}').rename(columns={f'{i}':'value'}) for i in x_groups]

numeric_df = pd.concat(numeric_dfs, axis=0)


px.histogram(numeric_df, x='value', nbins=1000, color = 'type', marginal='box', histnorm='probability density', barmode='overlay')

In [0]:
column_names = categorical_columns + numerical_columns
pr_df.get(column_names).isnull().sum()


In [0]:
# Density distributions all years
x_groups = ['earned_premium']
x = [pr_df.get(i) for i in x_groups]

(
    ff.create_distplot(x, x_groups)
    .update_layout(
        title='Density distributions for numerical columns',
        template = 'none')
)


##### NEW

In [0]:
# Transforming columns to 100 percent share
numeric_columns = ['incurred_amount',
                   'ultimate_amount',
                   'loss_reserve_amount',
                   'paid_amount',
                   'egpi',
                   'total_charges',
                   'cls_total_charges_orig',
                   'earned_premium',
                   'cls_total_incurred',
                   'expected_loss',             # pricing
                   'expected_premium',          # pricing
                   'external_expenses',         # pricing
                   'allocated_capital',         # pricing
                   'pricing_internal_costs',    # pricing
                   'expected_natcat_loss',      # pricing
                   'exp_shortfall_loss',        # pricing
                   'cat_inc',                   # pricing maybe
                   'nonnatcat_inc',
                   'earned_premium_omega',
                   'total_charges_omega',
                   'ultimate_amount_omega',
                   'incurred_amount_omega']



columns_to_select = (
    ['ceded_share', 'our_share'] + 
    [item for col in numeric_columns for item in [col, f'{col}_100', f'log_{col}_100']] + 
    ['retention', 'limit']
)

In [0]:
# Density distributions all years
x_groups = [ 'earned_premium', 'retention' , 'limit' ,'ultimate_amount']
x = [pr_df.get(i) for i in x_groups]



(
    ff.create_distplot(x, x_groups, bin_size=0.1)
    .update_layout(
        title='Density distributions for log earned premiums, ultimate amount and retention  <br><sup> Note: Notice how the "_100" values shift to the right, closer to retention </sup>',
        template = 'none')
)


In [0]:
# Density distributions all years
x_groups = ['log_retention', 'log_ultimate_amount_100', 'log_ultimate_amount', 'log_earned_premium', 'log_earned_premium_100']
x = [pr_df.query(f'{i} > 0').get(i).replace([np.inf, -np.inf], np.nan) for i in x_groups]



(
    ff.create_distplot(x, x_groups, bin_size=0.1)
    .update_layout(
        title='Density distributions for log earned premiums, ultimate amount and retention  <br><sup> Note: Notice how the "_100" values shift to the right, closer to retention </sup>',
        template = 'none')
)



In [0]:
# Density distributions by year 
years_list = list(range(2023, 2024))
numeric_columns = ['log_earned_premium_100']

x = [pr_df.query(f'uw_year == {i}').get(j).replace([np.inf, -np.inf], np.nan) for i in years_list for j in numeric_columns]
x_groups = [f'{i}_Y{j}' for i in numeric_columns for j in years_list ]

colors = ['#34495e', '#2ecc71', '#e74c3c']

(
    ff.create_distplot(x, x_groups, bin_size=0.1, show_hist=False, colors = colors)
    .update_layout(title='Density distributions for LOG earned premium amount by year')
    )

In [0]:
# Density distributions by year
years_list = list(range(2023, 2024))
numeric_columns = ['earned_premium_100']
x = [pr_df.query(f'uw_year == {i}').get(j).replace([np.inf, -np.inf], np.nan) for i in years_list for j in numeric_columns]
x_groups = [f'Year {j}' for j in years_list]

colors = ['#34495e', '#2ecc71', '#e74c3c']

(
    ff.create_distplot(x, x_groups, bin_size=0.1, show_hist=False, colors = colors)
    .update_layout(title='Density distributions for earned premium amount by year')
    )

In [0]:
# Density distributions by year
years_list = list(range(2023, 2024))
numeric_columns = ['egpi']
x = [df.query(f'uw_year == {i}').get(j).replace([np.inf, -np.inf], np.nan) for i in years_list for j in numeric_columns]
x_groups = [f'Year {j}' for j in years_list]

colors = ['#34495e', '#2ecc71', '#e74c3c']

(
    ff.create_distplot(x, x_groups, bin_size=0.1, show_hist=False, colors = colors)
    .update_layout(title='Density distributions for earned premium amount by year')
    )

In [0]:
# Density distributions by year
years_list = list(range(2023, 2024))
numeric_columns = ['log_ultimate_amount_100']
x = [df.query(f'uw_year == {i}').get(j).replace([np.inf, -np.inf], np.nan) for i in years_list for j in numeric_columns]
x_groups = [f'Year {j}' for i in numeric_columns for j in years_list ]

(
    ff.create_distplot(x, x_groups, bin_size=0.1, show_hist=False)
    .update_layout(title='Density distributions for LOG ultimate amount by year')
    )


In [0]:
# Density distributions by year
years_list = list(range(2023, 2024))
numeric_columns = ['ultimate_amount_100']
x = [df.query(f'uw_year == {i}').get(j).replace([np.inf, -np.inf], np.nan) for i in years_list for j in numeric_columns]
x_groups = [f'{i}_Y{j}' for i in numeric_columns for j in years_list ]

(
    ff.create_distplot(x, x_groups, bin_size=0.1, show_hist=False)
    .update_layout(title='Density distributions for ultimate amount by year')
    )


In [0]:
# Normalizing columns
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

df_2022 = df.query('uw_year == 2022')
df_2023 = df.query('uw_year == 2023')
df_2024 = df.query('uw_year == 2024')

df_2022[['log_retention_scaled', 'log_ultimate_amount_100_scaled', 'log_earned_premium_100_scaled']] = scaler.fit_transform(df_2022.get(['log_retention', 'log_ultimate_amount_100', 'log_earned_premium_100']))

df_2023[['log_retention_scaled', 'log_ultimate_amount_100_scaled', 'log_earned_premium_100_scaled']] = scaler.fit_transform(df_2023.get(['log_retention', 'log_ultimate_amount_100', 'log_earned_premium_100']))

df_2024[['log_retention_scaled', 'log_ultimate_amount_100_scaled', 'log_earned_premium_100_scaled']] = scaler.fit_transform(df_2024.get(['log_retention', 'log_ultimate_amount_100', 'log_earned_premium_100']))


In [0]:
# Density distributions of scaled columns by year
numeric_columns = ['log_retention_scaled', 'log_ultimate_amount_100_scaled', 'log_earned_premium_100_scaled']
x = [*[df_2022.get(i) for i in numeric_columns], *[df_2023.get(i) for i in numeric_columns], *[df_2024.get(i) for i in numeric_columns]]

x_groups = [f'{i}_{j}'  for j in [2022, 2023, 2024] for i in numeric_columns]

ff.create_distplot(x, x_groups, bin_size=0.005)

In [0]:
df.get(['retention','ultimate_amount_100', 'log_retention','log_ultimate_amount', 'cat_inc', 'nonnatcat_inc']).pipe(display)

In [0]:
# Summary of numeric columns
pd.set_option('display.float_format', lambda x: '%.3f' % x)

these_numeric_columns = [
    'ultimate_amount', 
  #  'total_charges', 
    'earned_premium',  
    'loss_ratio',
    'underwriting_ratio',
    'log_ultimate_amount',
    'log_earned_premium', 
    'log_ultimate_amount_100',
    'log_earned_premium_100', 
    'log_loss_ratio',
    'log_retention',
]

(
    pr_df
    .get([*these_numeric_columns])
    .describe(percentiles = [0.10, 0.25, 0.5, 0.75, 0.99])
)


##### 3.0 Ultimate amount EDA

In [0]:
# 'Histogram of Ultimate Amount
fig = (
    px.histogram(pr_df, x='ultimate_amount', color='uw_year', barmode ='overlay', opacity =0.8, marginal='box')
    .update_layout(
        title_text='Histogram of Ultimate Amount', # title of plot
        xaxis_title_text='Ultimate amount', # xaxis label
        yaxis_title_text='Count',
        showlegend=False)
)
fig.show()

In [0]:
# 'Histogram of Ultimate Amount
fig = (
    px.histogram(pr_df, x='ultimate_amount', color='uw_year', log_y=True, barmode ='overlay', opacity =0.8, marginal='box')
    .update_layout(
        title_text="Histogram of Ultimate Amount<br><sup>Log scale in y axis</sup>", # title of plot 
        xaxis_title_text='Ultimate amount', # xaxis label
        yaxis_title_text='Count  (log scale)',
        showlegend=False)
)
fig.show()


In [0]:
fig.write_image("/Workspace/Users/daguilarromero@scor.com/Data_science_project/master_thesis_files/fig1.svg")

In [0]:
# 'Histogram of Ultimate Amount (After filtering)
pr_df_filtered = pr_df.query('ultimate_amount <= 1000000') # 1 million
(
  px.histogram(pr_df_filtered, x='ultimate_amount', color='uw_year', log_y=True, barmode ='overlay', opacity =0.9, marginal='box')
  .update_layout(
    title_text='Histogram of Ultimate Amount (After filtering)', # title of plot
    xaxis_title_text='Ultimate amount', # xaxis label
    yaxis_title_text='Count  (log scale)', 
    legend_title_text='Underwriting year', 
    showlegend=False)
  )

In [0]:
# Density distribution for Log Ultimate Amount

x = [
    pr_df.get('log_ultimate_amount_100'),
    pr_df.get('log_earned_premium_100')
    ]

groups = ['log_ultimate_amount', 'log_earned_premium']

(
    ff.create_distplot(x, groups, bin_size=0.25)
    .update_layout(
        title_text='Density distribution for Log Ultimate Amount and Log Earned Premium', # title of plot
        xaxis_title_text='Log Ultimate amount', # xaxis label
        yaxis_title_text='Densities', 
        legend_title_text='Variable')
    )

In [0]:
# Creating dataframe that will be used for parallel categories plots
pr_df_bins = (
    pr_df
    .assign(log_ultimate_amount_bins = lambda x: pd.cut(x.log_ultimate_amount, bins=[0,5,6,7,8,9,10,11,12,13,14, 15, x.log_ultimate_amount_100.max()]).astype(str))
)

In [0]:
px.parallel_categories(pr_df_bins.sort_values('log_ultimate_amount_bins'), dimensions=['uw_year', 'ri_region', 'lobn2', 'log_ultimate_amount_bins'])


In [0]:
# Boxplots for Log Ultimate Amount by lobn1 and colored by Main Nature
print('Notice how the distributions for Non Prop have lower values')

(
    px.box(data_frame = pr_df, x = 'lobn2', y = 'log_ultimate_amount')
    .update_xaxes(matches = None, tickangle=270)
    .update_layout(
        title_text= f'Boxplots for Log Ultimate Amount by lobn1 and colored by Main Nature', # title of plot
        xaxis_title_text='LobN1', # xaxis label
        yaxis_title_text='Log Ultimate Amount')
    )


In [0]:
# Boxplots for Log Ultimate Amount by lobn1 and colored by Main Nature
print('Notice how the distributions for Non Prop have lower values')

(
    px.box(data_frame = pr_df, x = 'lobn1', y = 'ultimate_amount')
    .update_xaxes(matches = None, tickangle=270)
    .update_layout(
        title_text= f'Boxplots for Ultimate Amount by lobn1 and colored by Main Nature', # title of plot
        xaxis_title_text='LobN1', # xaxis label
        yaxis_title_text='Ultimate Amount')
    )

In [0]:
# Creating dataframe that will be used for FILTERED boxplots

x_value = ['lobn2']
y_value = 'log_ultimate_amount'
these_main_nature_name = ['Non Prop.']
by_this_region = pr_df.ri_region.unique()[1]

this_data = (
    pr_df
  #  .query(f'ri_region == "{by_this_region}"')
    .assign(average = lambda x: x.groupby(x_value).log_ultimate_amount.transform('median'))
    .sort_values(['average', 'main_nature_name'])
    )



In [0]:
# Boxplots for Log Ultimate Amount by lobn1 and colored by Main Nature

(
    px.violin(data_frame = this_data, x = x_value, y = 'log_ultimate_amount', points='all')
    .update_xaxes(matches = None, tickangle=270)
    .update_layout(
        title_text= f'Violin plots and points for Log Ultimate Amount by lobn1 and colored by Main Nature for {by_this_region}', # title of plot
        xaxis_title_text='Line of Business', # xaxis label
        yaxis_title_text='Log Ultimate Amount')
    )



In [0]:
(
    px.box(data_frame = this_data, x = x_value, y = y_value, color='ri_region')
    .update_xaxes(matches = None, tickangle=270)
    .update_layout(
        title_text='Boxplots for Log Ultimate Amount by Line of Business and colored by Region', # title of plot
        xaxis_title_text='Line of Business', # xaxis label
        yaxis_title_text='Log Ultimate Amount', 
        legend_title_text='Region')
    )


In [0]:

# Boxplots for Log Ultimate Amount by lobn1 and faceted by Main Nature 
(
    px.box(data_frame = this_data, x = x_value, y = y_value)
    .update_xaxes(matches = None, tickangle=270)
    .update_layout(
        title_text='Boxplots for Log Ultimate Amount by lobn1', # title of plot
        xaxis_title_text='LobN1', # xaxis label
        yaxis_title_text='Log Ultimate Amount')
    .for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    )

In [0]:
df_to_plot = (
    pr_df
 #   .query('log_retention > 5')
 #   .query('log_retention_plus_limit > 5')
    .sample(n = 10, random_state=42)
    .assign(omega_contract_and_section = lambda x: x.contract_omega_id.astype(str) + '-' + x.omega_section.astype(str),
            retention_plus_limit = lambda x: x.retention + x.limit,
            retention_plus_ultimate_amount = lambda x: x.retention + x.ultimate_amount)
    .get(['uw_year', 'omega_contract_and_section', 'retention', 'retention_plus_limit', 'retention_plus_ultimate_amount'])
    .melt(id_vars = ['uw_year', 'omega_contract_and_section'],
          value_vars = ['retention', 'retention_plus_limit', 'retention_plus_ultimate_amount'],
          var_name = 'variable', 
          value_name = 'value')
)

df_to_plot.variable = pd.Categorical(df_to_plot.variable, 
                      categories=[ 'retention', 'retention_plus_ultimate_amount', 'retention_plus_limit'],
                      ordered=True)

df_to_plot =  df_to_plot.sort_values(['variable', 'value'])

px.scatter(df_to_plot, x = 'omega_contract_and_section', y='value',color = 'variable', log_y=True)

In [0]:
df_to_plot = (
    pr_df
 #   .query('log_retention > 5')
 #   .query('log_retention_plus_limit > 5')
    .sample(n = 100, random_state=42)
    .assign(omega_contract_and_section = lambda x: x.contract_omega_id.astype(str) + '-' + x.omega_section.astype(str))
    .get(['uw_year', 'omega_contract_and_section', 'retention', 'ultimate_amount'])
    .melt(id_vars = ['uw_year', 'omega_contract_and_section'],
          value_vars = ['retention', 'ultimate_amount'],
          var_name = 'variable', 
          value_name = 'value')
)

df_to_plot.variable = pd.Categorical(df_to_plot.variable, 
                      categories=[ 'ultimate_amount', 'retention' ],
                      ordered=True)

df_to_plot =  df_to_plot.sort_values(['variable', 'value'])

px.scatter(df_to_plot, x = 'omega_contract_and_section', y='value',color = 'variable', log_y=True)

##### 4.0 Clustering Retention and Limits

In [0]:
df_to_cluster = (
    pr_df
    .query('log_earned_premium_100 > 5')
    .query('log_retention > 1 ')
 #   .assign(log_earned_premium = lambda x: np.log(x.earned_premium))
   # .sample(n = 1000, random_state=42)
)

#px.scatter(df_to_cluster, x = 'log_retention', y='log_retention_plus_limit')
px.scatter(df_to_cluster, x = 'log_earned_premium_100', y='log_retention', color='log_ultimate_amount')

In [0]:
from sklearn.model_selection import train_test_split

X = df_to_cluster.get(['log_retention', 'log_earned_premium_100'])
y = df_to_cluster.log_ultimate_amount

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [0]:
from sklearn import preprocessing

X_train_norm = preprocessing.normalize(X_train)
X_test_norm = preprocessing.normalize(X_test)

In [0]:
from sklearn.cluster import KMeans
import seaborn as sns

kmeans = KMeans(n_clusters = 3, random_state = 42)
kmeans.fit(X_train_norm)

sns.scatterplot(data = X_train,  x = 'log_earned_premium_100', y='log_retention', hue = kmeans.labels_)

In [0]:
sns.boxplot(x = kmeans.labels_, y = y_train)

In [0]:
labels_dict = {}
sse_dict = {}

for k in range(2, 20):
    # train the model for current value of k on training data
    kmeans =  KMeans(n_clusters = k, random_state = 42).fit(X_train_norm)
    
    # append the model to fits
    labels_dict[k] = kmeans.labels_

    # Append the silhouette score to scores
    sse_dict[k] = kmeans.inertia_

In [0]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(list(sse_dict.keys()), list(sse_dict.values()))
plt.xlabel("Number of cluster")
plt.ylabel("SSE")
plt.show()

In [0]:
kmeans = KMeans(n_clusters = 5, random_state = 42)
kmeans.fit(X_train_norm)


(
    sns.scatterplot(data = X_train,  x = 'log_earned_premium_100', y='log_retention', hue = kmeans.labels_, palette='Set2')
    .set_title("k=5 mean clustering for log_retention and log_earned_premium")

    
    )

In [0]:
sns.boxplot(x = kmeans.labels_, y = y_train)

In [0]:
df_to_plot = pd.DataFrame({
    'log_earned_premium_100': X_train.log_earned_premium_100,
    'log_retention': X_train.log_retention,
 #   'log_retention_plus_limit': X_train.log_retention_plus_limit,
    'log_ultimate_amount': y_train,
    'cluster': kmeans.labels_
})


px.scatter(df_to_plot, x = 'log_earned_premium_100', y='log_retention', color = 'cluster')

##### 5.0 Dataframes of frequency statistics

In [0]:
# Creating the Dictionary of Dataframes of frequency statistics

# list of columns of type string
string_type_columns_list = pr_df.select_dtypes('object').columns.tolist()

# Dictionary of Dataframes of frequency statistics
string_column_frequency_dict = {
    col: (
        pr_df
        .groupby(['uw_year',col], as_index=False)
        .agg(n =(f'{col}','count'),
             ultimate_amount = ('ultimate_amount','sum'))
        .sort_values(['uw_year','n'], ascending=[True, False])
        .assign(
            n_pct =             lambda x: x.n / x.groupby('uw_year').n.transform('sum'),
            n_accumulated_pct = lambda x: x.groupby('uw_year').n_pct.transform('cumsum'),
            ult_amount_pct =    lambda x: x.ultimate_amount / x.groupby('uw_year').ultimate_amount.transform('sum'),
            ult_amount_accumulated_pct = lambda x: x.groupby('uw_year').ult_amount_pct.transform('cumsum'),
            log_ultimate_amount = lambda x: np.log(x.ultimate_amount)
            )
        .reset_index(drop=True)
        .get(['uw_year', col, 'n', 'n_pct', 'n_accumulated_pct', 'ultimate_amount', 'ult_amount_pct', 'ult_amount_accumulated_pct', 'log_ultimate_amount'])
    ) 
    for col in string_type_columns_list
}


In [0]:
# Printing of Dataframes of frequency statistics
start = 0
end  = 9

# 2.3 category frequency for ALL string columns
for k in list(string_column_frequency_dict.keys())[start:end]:
    # 1.0 Prints index number
    print('index number: ', list(string_column_frequency_dict.keys()).index(k)) 
    # 2.0 Prints number of categories (rows) per categorial column
    print('n categories: ', string_column_frequency_dict[k].shape[0])                 
    # 3.0 Prints first 20 rows oF frequency table for categorial column
    print(string_column_frequency_dict[k].to_string())
    print("")               

##### 5.0 Summary of data structure after selecting columns

In [0]:
these_columns = [
    'primary_key',
    'quarter', 
    'uw_year',
    'contract_omega_id',
    'omega_section',
    'inception_date',
    'expiry_date',
    'data_type',
    'omega_region', 
    'ri_region',
    'ri_area',
    'geo_level', 
    'geon1',
    'lobn1',
    'lobn2', 
    'main_nature_name',
    'nature_name', 
    'per_event', 
    'per_risk', 
    'omega_group_division_org', 
    'program_name', 
    'program_bouquet_contract', 
    'bouquet_number',
   # 'bouquet_name'
    'program_bouquet_contract_id', 
    'company_name', 
    'reporting_country', 
    'subsidiary_name', 
    'lob_name', 
    'sob_name', 
    'top_name', 
    'guarantee', 
    'accounting_admin_type_label', 
 # 'company_size', 
 'intermediary_name', 
 'intermediary_group', 
 'intermediary_ult_group', 
 'client_segmentation_assessment', 
 # 'uw_portfolios', 
 'us_pc_cancelled_flag', 
 'gcu_flag',
 'ultimate_id',
 'ledger_name', 
 'guarantee', 
 'portfolio_origin',
 'ultimate_amount',
 'log_ultimate_amount'
    ]



In [0]:

pr_df_2 = pr_df.get(these_columns).sort_values(['primary_key'])


print(pr_df.dtypes.value_counts())
print('')
print(pr_df_2.dtypes.value_counts())


In [0]:
this_list = ['ri_region', 'ri_area', 'reporting_country']
this_list = ['lobn2', 'lobn1']
#this_list = ['company_name','subsidiary_name']
#this_list = ['per_event', 'per_risk']
pr_df.get(this_list).value_counts().reset_index().sort_values(this_list).pipe(display)

In [0]:
pr_df.reporting_country

##### 99.0 Sandbox

In [0]:
# Correlation ratio function
def correlation_ratio(df, category_col, value_col):

    """
    Calculate the correlation ratio (η²) for a categorical and a continuous variable using pandas.
    
    Parameters:
    - df: pandas DataFrame containing the data
    - category_col: str, column name for the categorical variable
    - value_col: str, column name for the continuous variable
    
    Returns:
    - eta_squared: float, the correlation ratio
    """
    # Overall mean of the continuous variable
    overall_mean = df[value_col].mean()
    
    # Group by the categorical column and calculate group means and sizes
    grouped = df.groupby(category_col)[value_col].agg(['mean', 'count'])
    
    # Between-group sum of squares
    ss_between = ((grouped['count'] * (grouped['mean'] - overall_mean)**2).sum())
    
    # Total sum of squares
    ss_total = ((df[value_col] - overall_mean)**2).sum()
    
    # Correlation ratio (η²)
    eta_squared = ss_between / ss_total
    return float(eta_squared**(1/2))


In [0]:
test_df = pr_df.assign(super_feature = lambda x: x.main_nature_name + '_' + x.ri_region + '_' + x.lobn1)

columns = list(test_df.select_dtypes(include=['object']).columns)
correlation_ratio_list = [correlation_ratio(test_df, x, 'log_ultimate_amount_100') for x in columns ]

(
    pd.DataFrame({'column': columns, 'correlation_ratio': correlation_ratio_list})
    .sort_values(by='correlation_ratio', ascending=False)
    .pipe(display)
)

In [0]:
test_df.company_name.value_counts().to_frame().sort_values(by='company_name', ascending=False).head(50).reset_index().rename(columns={'index': 'company_name', 'company_name': 'count'})

In [0]:
numerical_value = 'log_ultimate_amount_100'

df_to_dummify = (
    pr_df
    .query('uw_year == 2023')
    .get(['main_nature_name', 'ri_region', 'lobn1', numerical_value])
)

numerical_column = df_to_dummify[numerical_value]

dummified_df = (
    df_to_dummify
    .drop(numerical_value, axis =1)
    .pipe(pd.get_dummies)
)

result_df = pd.concat([dummified_df, numerical_column], axis = 1)

correlation_ratio_list = [correlation_ratio(result_df, x, numerical_value) for x in dummified_df.columns]

pd.DataFrame({'column': dummified_df.columns, 'correlation_ratio': correlation_ratio_list}).sort_values(by='correlation_ratio', ascending=False).pipe(display)